In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [9]:
#dir(pd)
#pd.__all__
from inspect import getmembers, isfunction
print(getmembers(pd), isfunction)

[('ArrowDtype', <class 'pandas.core.dtypes.dtypes.ArrowDtype'>), ('BooleanDtype', <class 'pandas.core.arrays.boolean.BooleanDtype'>), ('Categorical', <class 'pandas.core.arrays.categorical.Categorical'>), ('CategoricalDtype', <class 'pandas.core.dtypes.dtypes.CategoricalDtype'>), ('CategoricalIndex', <class 'pandas.core.indexes.category.CategoricalIndex'>), ('DataFrame', <class 'pandas.core.frame.DataFrame'>), ('DateOffset', <class 'pandas._libs.tslibs.offsets.DateOffset'>), ('DatetimeIndex', <class 'pandas.core.indexes.datetimes.DatetimeIndex'>), ('DatetimeTZDtype', <class 'pandas.core.dtypes.dtypes.DatetimeTZDtype'>), ('ExcelFile', <class 'pandas.io.excel._base.ExcelFile'>), ('ExcelWriter', <class 'pandas.io.excel._base.ExcelWriter'>), ('Flags', <class 'pandas.core.flags.Flags'>), ('Float32Dtype', <class 'pandas.core.arrays.floating.Float32Dtype'>), ('Float64Dtype', <class 'pandas.core.arrays.floating.Float64Dtype'>), ('Grouper', <class 'pandas.core.groupby.grouper.Grouper'>), ('HDFS

In [42]:
df= pd.read_csv("london_merged.csv")
df.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [43]:
df['timestamp']= pd.to_datetime(df['timestamp'])
df['hour']= df['timestamp'].dt.hour
df['year']= df['timestamp'].dt.year
df['day_of_week']= df['timestamp'].dt.dayofweek
df['month']= df['timestamp'].dt.month
df.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,hour,year,day_of_week,month
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0,0,2015,6,1
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0,1,2015,6,1
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0,2,2015,6,1
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0,3,2015,6,1
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0,4,2015,6,1


In [44]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   timestamp     17414 non-null  datetime64[ns]
 1   cnt           17414 non-null  int64         
 2   t1            17414 non-null  float64       
 3   t2            17414 non-null  float64       
 4   hum           17414 non-null  float64       
 5   wind_speed    17414 non-null  float64       
 6   weather_code  17414 non-null  float64       
 7   is_holiday    17414 non-null  float64       
 8   is_weekend    17414 non-null  float64       
 9   season        17414 non-null  float64       
 10  hour          17414 non-null  int32         
 11  year          17414 non-null  int32         
 12  day_of_week   17414 non-null  int32         
 13  month         17414 non-null  int32         
dtypes: datetime64[ns](1), float64(8), int32(4), int64(1)
memory usage: 1.6 MB
None


In [45]:
#preprocessing the data
missing_values= df.isnull().sum()
print(missing_values)

timestamp       0
cnt             0
t1              0
t2              0
hum             0
wind_speed      0
weather_code    0
is_holiday      0
is_weekend      0
season          0
hour            0
year            0
day_of_week     0
month           0
dtype: int64


In [46]:
#remove or fill the missing values if necessary
df['t1'].fillna(df['t1'].mean(), inplace= True)
df['t2'].fillna(df['t2'].mean(), inplace= True)
df['hum'].fillna(df['hum'].mean(), inplace= True)
df['wind_speed'].fillna(df['wind_speed'].mean(), inplace= True)
df['weather_code'].fillna(method= 'ffill', inplace= True)

C:\Users\HRISHITA\AppData\Local\Temp\ipykernel_8456\855255392.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['weather_code'].fillna(method= 'ffill', inplace= True)


In [47]:
threshold= df['cnt'].mean()
df['user_type']= (df['cnt'] > threshold).astype(int)

In [48]:
df= df.drop(columns= ['cnt', 'timestamp'])

X= df.drop('user_type', axis=1)
y= df['user_type']

df

,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,hour,year,day_of_week,month,user_type
0,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0,0,2015,6,1,0
1,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0,1,2015,6,1,0
2,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0,2,2015,6,1,0
3,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0,3,2015,6,1,0
4,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0,4,2015,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17409,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0,19,2017,1,1,0
17410,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0,20,2017,1,1,0
17411,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0,21,2017,1,1,0
17412,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0,22,2017,1,1,0


In [49]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.3, random_state=0)

In [50]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [52]:
rf_model= RandomForestClassifier(n_estimators= 100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [55]:
rf_pred= rf_model.predict(X_test)

In [65]:
print("\nEvaluation:\n")
print("Accuracy=%.4f"%accuracy_score(y_test, rf_pred))
print("Precision=%.4f"%precision_score(y_test, rf_pred))
print("F1-score=%.4f"%f1_score(y_test, rf_pred))
print("Recall=%.4f"%recall_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred))


Evaluation:

Accuracy=0.9489
Precision=0.9253
F1-score=0.9353
Recall=0.9456
              precision    recall  f1-score   support

           0       0.96      0.95      0.96      3183
           1       0.93      0.95      0.94      2042

    accuracy                           0.95      5225
   macro avg       0.94      0.95      0.95      5225
weighted avg       0.95      0.95      0.95      5225

